In [45]:
import glob,sys
from pathlib import Path
pinkRig_path= glob.glob(r'C:\Users\*\Documents\Github\PinkRigs')
pinkRig_path = Path(pinkRig_path[0])
sys.path.insert(0, (pinkRig_path.__str__()))
kilofolder = __import__("Analysis.+kilo.python_.convert_to_ibl_format") # this is necessary because some wise people decided to write a package with python and matlab in it. #wisdom.
kilo = getattr(kilofolder, "+kilo")
add_anat_to_ibl_format = kilo.python_.convert_to_ibl_format.add_anat_to_ibl_format

# Path to raw ephys data
#ephys_path = Path(r'Z:\AV014\2022-07-07\ephys\AV014_2022-07-07_NatImagesSparseNoise_g0\AV014_2022-07-07_NatImagesSparseNoise_g0_imec0')
     
#add_anat_to_ibl_format(ephys_path,ks_folder='pyKS',recompute=True)

In [48]:
# add a func to loop over SC recordings (selected somehow by Flora, what a shame we cannot pay for someone competent.)

import pandas as pd 
from Admin.csv_pyhandlers import get_server_location
from Analysis.helpers.queryExp import load_data

prPath = get_server_location()
session_list = pd.read_csv(prPath / 'Helpers/sc_selected_recordings.csv')


for _,r in session_list.iterrows():
    recdat = load_data(subject = r.Subject,expDate=r.expDate, expNum = r.expNum)
    # now I will try to convert all files that were already converted to ibl format
    ephys_path = Path(recdat.iloc[0].expFolder).parent / 'ephys'
    ephys_recordings = list(ephys_path.glob('**\pyKS\output\ibl_format\channels.localCoordinates.npy'))

    for rec in ephys_recordings: 
            s=add_anat_to_ibl_format(rec.parents[3],recompute= True)
            print(rec)

converting data to IBL format ...


c:\Users\Experiment\anaconda3\envs\iblenv\lib\site-packages\phylib\io\model.py:1053: RuntimeWarning: invalid value encountered in true_divide
  spikes_depths[ispi] = (np.sum(np.transpose(ypos * features) /
Extracting waveforms: 100%|██████████| 4806.6568/4806.6568 [00:13<00:00, 368.31it/s]
Converting to ALF:  84%|████████▍ | 105/125 [00:18<00:04,  4.72it/s]c:\Users\Experiment\anaconda3\envs\iblenv\lib\site-packages\phylib\io\model.py:1053: RuntimeWarning: invalid value encountered in true_divide
  spikes_depths[ispi] = (np.sum(np.transpose(ypos * features) /
Converting to ALF: 145it [00:26,  5.57it/s]                         
2023-01-11 17:48:56.751 INFO     [extract_files.py:82] Computing QC for \\zinu.cortexlab.net\Subjects\AV005\2022-05-27\ephys\AV005_2022-05-27_ActivePassive_g0\AV005_2022-05-27_ActivePassive_g0_imec0\AV005_2022-05-27_ActivePassive_g0_t0.imec0.ap.cbin


the probe/shank IDs do not match.████████████████████████████████████████████████████████████████████-| 99.3% 
the probe/shank IDs do not match.
the probe/shank IDs do not match.
the probe/shank IDs do not match.
\\zinu.cortexlab.net\Subjects\AV005\2022-05-27\ephys\AV005_2022-05-27_ActivePassive_g0\AV005_2022-05-27_ActivePassive_g0_imec0\pyKS\output\ibl_format\channels.localCoordinates.npy
converting data to IBL format ...


Extracting waveforms: 100%|██████████| 1790.7910333333334/1790.7910333333334 [00:06<00:00, 266.11it/s]
Converting to ALF: 145it [00:08, 16.67it/s]                         
2023-01-11 18:37:22.324 INFO     [extract_files.py:82] Computing QC for \\zinu.cortexlab.net\Subjects\AV005\2022-05-27\ephys\AV005_2022-05-27_NatImagesSparseNoise_g0\AV005_2022-05-27_NatImagesSparseNoise_g0_imec0\AV005_2022-05-27_NatImagesSparseNoise_g0_t0.imec0.ap.cbin


the probe/shank IDs do not match.████████████████████████████████████████████████████████████████████-| 99.0% 
the probe/shank IDs do not match.
the probe/shank IDs do not match.
the probe/shank IDs do not match.
\\zinu.cortexlab.net\Subjects\AV005\2022-05-27\ephys\AV005_2022-05-27_NatImagesSparseNoise_g0\AV005_2022-05-27_NatImagesSparseNoise_g0_imec0\pyKS\output\ibl_format\channels.localCoordinates.npy
converting data to IBL format ...


...

Extracting waveforms: 100%|██████████| 929.8904333333334/929.8904333333334 [00:11<00:00, 78.84it/s]
Converting to ALF:  84%|████████▍ | 105/125 [00:17<00:03,  5.24it/s]c:\Users\Experiment\anaconda3\envs\iblenv\lib\site-packages\phylib\io\model.py:1053: RuntimeWarning: invalid value encountered in true_divide
  spikes_depths[ispi] = (np.sum(np.transpose(ypos * features) /
Converting to ALF: 145it [00:20,  7.19it/s]                         
2023-01-11 22:21:28.489 INFO     [extract_files.py:82] Computing QC for \\zinu.cortexlab.net\Subjects\AV005\2022-05-12\ephys\AV005_2022-05-12_Spontaneous_Part3_g0\AV005_2022-05-12_Spontaneous_Part3_g0_imec0\AV005_2022-05-12_Spontaneous_Part3_g0_t0.imec0.ap.cbin


 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
the probe/shank IDs do not match.
\\zinu.cortexlab.net\Subjects\AV005\2022-05-12\ephys\AV005_2022-05-12_Spontaneous_Part3_g0\AV005_2022-05-12_Spontaneous_Part3_g0_imec0\pyKS\output\ibl_format\channels.localCoordinates.npy
converting data to IBL format ...


Extracting waveforms: 100%|██████████| 745.5411333333333/745.5411333333333 [00:10<00:00, 73.33it/s]
Converting to ALF:  84%|████████▍ | 105/125 [00:13<00:02,  7.06it/s]c:\Users\Experiment\anaconda3\envs\iblenv\lib\site-packages\phylib\io\model.py:1053: RuntimeWarning: invalid value encountered in true_divide
  spikes_depths[ispi] = (np.sum(np.transpose(ypos * features) /
Converting to ALF: 145it [00:14,  9.71it/s]                         
2023-01-11 22:30:57.892 INFO     [extract_files.py:82] Computing QC for \\zinu.cortexlab.net\Subjects\AV005\2022-05-12\ephys\AV005_2022-05-12_Spontaneous_Part4_g0\AV005_2022-05-12_Spontaneous_Part4_g0_imec0\AV005_2022-05-12_Spontaneous_Part4_g0_t0.imec0.ap.cbin


 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
the probe/shank IDs do not match.
\\zinu.cortexlab.net\Subjects\AV005\2022-05-12\ephys\AV005_2022-05-12_Spontaneous_Part4_g0\AV005_2022-05-12_Spontaneous_Part4_g0_imec0\pyKS\output\ibl_format\channels.localCoordinates.npy
converting data to IBL format ...


c:\Users\Experiment\anaconda3\envs\iblenv\lib\site-packages\phylib\io\model.py:1053: RuntimeWarning: invalid value encountered in true_divide
  spikes_depths[ispi] = (np.sum(np.transpose(ypos * features) /
Extracting waveforms: 100%|██████████| 5956.501666666667/5956.501666666667 [00:09<00:00, 607.34it/s]
Converting to ALF:  84%|████████▍ | 105/125 [00:13<00:03,  6.66it/s]c:\Users\Experiment\anaconda3\envs\iblenv\lib\site-packages\phylib\io\model.py:1053: RuntimeWarning: invalid value encountered in true_divide
  spikes_depths[ispi] = (np.sum(np.transpose(ypos * features) /
Converting to ALF: 145it [00:16,  8.60it/s]                         
2023-01-11 22:39:18.40 INFO     [extract_files.py:82] Computing QC for \\zinu.cortexlab.net\Subjects\AV006\2022-03-28\ephys\AV006_2022-03-28_ActivePassive_g0\AV006_2022-03-28_ActivePassive_g0_imec0\AV006_2022-03-28_ActivePassive_g0_t0.imec0.ap.cbin
